In [167]:
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import pandas as pd
import pdb

In [2]:
t = torch.rand(4, 4, 5)

In [3]:
t.view(4, 10, -1)


(0 ,.,.) = 
  0.6916  0.5430
  0.0157  0.3273
  0.5972  0.0859
  0.9671  0.7922
  0.7415  0.5100
  0.9596  0.6191
  0.3680  0.9812
  0.7328  0.8542
  0.8676  0.3984
  0.3522  0.0929

(1 ,.,.) = 
  0.6525  0.6418
  0.2679  0.8705
  0.4903  0.6075
  0.5642  0.8171
  0.0773  0.2623
  0.8090  0.2616
  0.7758  0.3871
  0.3868  0.7190
  0.5472  0.4373
  0.4554  0.0949

(2 ,.,.) = 
  0.8723  0.0695
  0.8425  0.3667
  0.6846  0.4431
  0.6133  0.4877
  0.1150  0.9401
  0.1688  0.9180
  0.1107  0.0581
  0.3840  0.8331
  0.6737  0.6934
  0.8806  0.3097

(3 ,.,.) = 
  0.5221  0.4753
  0.7044  0.7149
  0.8597  0.4956
  0.0169  0.0460
  0.0095  0.1659
  0.2617  0.4398
  0.9713  0.9629
  0.8688  0.0127
  0.6088  0.5575
  0.8656  0.9895
[torch.FloatTensor of size 4x10x2]

In [6]:
torch.ones(5, 3).numpy()

array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]], dtype=float32)

In [9]:
torch.from_numpy(np.array([4, 5, 3]))


 4
 5
 3
[torch.LongTensor of size 3]

In [18]:
x = torch.ones(2, 2)

In [19]:
x


 1  1
 1  1
[torch.FloatTensor of size 2x2]

In [20]:
y = x+2

In [21]:
z = y*y*3

In [22]:
out = z.mean()

In [24]:
out.backward()

AttributeError: 'float' object has no attribute 'backward'

In [27]:
torch.autograd(y)

TypeError: 'module' object is not callable

In [260]:
class BigData(Dataset):
    def __init__(self, df, dep, num_sites):
        self.length = max(df['day']) + 1
        self.num_sites = num_sites
        self.array = torch.Tensor(df.drop(columns=dep).values)
        self.index = torch.Tensor(df['day'].values)
        self.y = torch.Tensor(df[dep].values)
        
    def __len__(self):
        return self.length
    
    def __getitem__(self, idx):
        ind = [self.index == idx]
        mask = torch.zeros(self.array.shape)
        for i in range(self.array.size(1)):
            mask[:, i] = ind[0]
        sample = self.array.masked_select(mask.byte()).view(-1, self.array.size(1))
        y = self.y.masked_select(ind[0].byte())
        return sample, y

In [261]:
df = pd.DataFrame(columns=['site', 'day', 'prod'])
df['site'] = [1, 1, 1, 1, 2, 2, 3, 3, 3]
df['day'] = [0, 1, 2, 3, 0, 1, 0, 1, 2]
df['prod'] = [33.4, 14.5, 31.3, 45.2, 64.3, 41.7, 13.6, 17.6, 19.5]

In [262]:
dataset = BigData(df, 'prod', 3)

In [263]:
DL = DataLoader(dataset, batch_size=1, shuffle=False)

In [264]:
for i, batch in enumerate(DL):
    print(i, batch)

0 [
(0 ,.,.) = 
  1  0
  2  0
  3  0
[torch.FloatTensor of size 1x3x2]
, 
 33.4000  64.3000  13.6000
[torch.FloatTensor of size 1x3]
]
1 [
(0 ,.,.) = 
  1  1
  2  1
  3  1
[torch.FloatTensor of size 1x3x2]
, 
 14.5000  41.7000  17.6000
[torch.FloatTensor of size 1x3]
]
2 [
(0 ,.,.) = 
  1  2
  3  2
[torch.FloatTensor of size 1x2x2]
, 
 31.3000  19.5000
[torch.FloatTensor of size 1x2]
]
3 [
(0 ,.,.) = 
  1  3
[torch.FloatTensor of size 1x1x2]
, 
 45.2000
[torch.FloatTensor of size 1x1]
]


In [214]:
next(iter(DL))


(0 ,.,.) = 
   1.0000   0.0000  33.4000
   2.0000   0.0000  64.3000
   3.0000   0.0000  13.6000
[torch.FloatTensor of size 1x3x3]